# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

In [4]:
import pandas as pd
import sqlite3

In [7]:
conn = sqlite3.connect('enron.db')
c = conn.cursor()

In [8]:
results_employee = c.execute('SELECT * FROM EmployeeBase LIMIT 5;')
results_employee.fetchall()

[(1,
  u'John Arnold',
  u'Forestry',
  u'ENA Gas Financial',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (2,
  u'Harry Arora',
  u'Forestry',
  u'ENA East Power',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (3,
  u'Robert Badeer',
  u'Forestry',
  u'ENA West Power',
  u'Mgr Trading',
  u'Male',
  u'Junior'),
 (4,
  u'Susan Bailey',
  u'Legal',
  u'ENA Legal',
  u'Specialist Legal',
  u'Female',
  u'Junior'),
 (5,
  u'Eric Bass',
  u'Forestry',
  u'ENA Gas Texas',
  u'Trader',
  u'Male',
  u'Junior')]

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [15]:
recipient_message = c.execute("SELECT * FROM MessageBase LIMIT 5;")
recipient_message.fetchall()

[(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138),
 (2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138),
 (3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138),
 (4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138),
 (5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)]

Import each of the 3 tables to a Pandas Dataframes

In [19]:
employees = pd.read_sql('SELECT * FROM EmployeeBase;', con=conn)
employees.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [20]:
message = pd.read_sql('SELECT * FROM MessageBase;', con=conn)
message.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [21]:
recipient = pd.read_sql('SELECT * FROM RecipientBase;', con=conn)
recipient.head()

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [23]:
#1 How many employees are there?
employees.shape

In [25]:
#2 How many messages are there in the database?

message.count()

mid          21635
filename     21635
unix_time    21635
subject      21635
from_eid     21635
dtype: int64

In [28]:
#3 Convert the timestamp column in the messages. 
# When was the oldest message sent? And the newest?

message['date_time'] = pd.to_datetime(message["unix_time"], unit='s')
message.head()

,mid,filename,unix_time,subject,from_eid,date_time
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138,1998-11-13 04:07:00
1,2,taylor-m/sent/17,911459940,Indemnification,138,1998-11-19 07:19:00
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138,1998-11-19 08:24:00
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138,1998-11-24 02:23:00
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138,1998-11-30 03:22:00


In [30]:
# When was the oldest message sent? And the newest?
message.date_time.describe()

count                   21635
unique                  21309
top       1999-12-27 03:21:00
freq                        4
first     1998-11-13 04:07:00
last      2002-06-21 13:37:34
Name: date_time, dtype: object

In [35]:
message['date_time'].max(), message['date_time'].min()

(Timestamp('2002-06-21 13:37:34'), Timestamp('1998-11-13 04:07:00'))

In [36]:
#Some messages are sent to more than one recipient. 
# Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
message.mid.value_counts()

2047     1
2708     1
4743     1
19084    1
17037    1
21135    1
10896    1
8849     1
14994    1
12947    1
661      1
6758     1
6806     1
4759     1
19100    1
17053    1
21151    1
10912    1
8865     1
15010    1
6790     1
645      1
2692     1
12931    1
19052    1
17005    1
21103    1
10864    1
8817     1
14962    1
        ..
15693    1
9550     1
11599    1
17746    1
19795    1
5464     1
7513     1
1370     1
15677    1
3387     1
13596    1
1338     1
15645    1
9502     1
11551    1
17698    1
19747    1
5416     1
7465     1
1322     1
3371     1
13612    1
15661    1
9518     1
11567    1
17714    1
19763    1
5432     1
7481     1
2049     1
Name: mid, dtype: int64

In [ ]:
import numpy as np
recipient.pivot_table

Rescale to investigate the tail of the curve

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.